In [ ]:
## Testing
print("hello")

: 

In [ ]:
## Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Load data
raw_df = pd.read_parquet("../data/raw/online_retail.parquet",engine="fastparquet")


: 

In [ ]:
## Clean the raw data

: 

In [ ]:
## Exploratory Data Analysis - DO NOT INCLUDE IN PIPELINE
raw_df.head() ## 500K rows, which are line items for invoices.
raw_df.describe() ## Negative quantity and price metrics, likely refunds. Should be removed.
(raw_df["Price"] < 0).sum() ## 3 line items are negative, so will remove these.
raw_df.info() ## There are about 100K line items that are null.
raw_df.isnull().sum() ## Nulls are all in customer ID feature.
raw_df.shape # Shape before cleaning.


: 

In [ ]:
## Full dataset cleaning
full_df = raw_df[(raw_df["Price"] >= 0) & (raw_df["Quantity"] >= 0)].dropna(subset=["Customer ID"])
full_df.shape # Shape after cleaning. Looks like there were an additional 10K rows removed for negative price and quantity.

: 

In [ ]:
full_df.head(20)

: 

In [ ]:
## Group into customer-level dataframe

## First create lineitem_amount column
full_df["lineitem_amount"] = full_df["Quantity"] * full_df["Price"]
full_df.head(20)


: 

In [ ]:
## Get max date within the entire dataset as reference for 'today'
MAX_DATE = full_df["InvoiceDate"].max().normalize()
print(MAX_DATE)

: 

In [ ]:
## Check on results
full_df["Invoice"].str.startswith("C").describe() 
full_df["Invoice"].str.startswith("c").describe() ## According to this, no invoices were cancelled

: 

In [ ]:
## Create invoice df
invoice_df = (
    full_df
    .assign(InvoiceDate=full_df["InvoiceDate"].dt.normalize())
    .groupby(["Customer ID", "InvoiceDate"], as_index=False)
    .agg(
        monetary=("lineitem_amount", "sum")
        # Frequency=("Invoice", "nunique") # This captures multiple invoices for customer in same day, which we don't care about
    )
)


: 

In [ ]:
## Sort by customer, then date to help get interpurchase days
invoice_df = invoice_df.sort_values(["Customer ID", "InvoiceDate"])
invoice_df.head()

: 

In [ ]:
## Calculate interpurchase days
invoice_df["days_between_purchases"] = (
    invoice_df
    .groupby("Customer ID")["InvoiceDate"]
    .diff()
    .dt.days
)
invoice_df.head(20)

: 

In [ ]:
## Create customer df, with core aggregations
customer_df = (
    invoice_df
    .groupby("Customer ID")
    .agg(
        customer_id=("Customer ID", "first"),

        # Recency: days since most recent invoice
        recency=("InvoiceDate", lambda x: (MAX_DATE - x.max().normalize()).days),

        # Frequency: number of invoices
        frequency=("InvoiceDate", "count"),

        # Monetary: total spend
        monetary=("monetary", "sum"),

        # Median days between purchases
        median_purchase_days=("days_between_purchases", "median")
    )
    .reset_index(drop=True)
)

: 

In [ ]:
## For medians NaN, just put in ithe recency (only one purchase)
customer_df["median_purchase_days"] = (
    customer_df["median_purchase_days"]
    .fillna(customer_df["recency"])
)
customer_df.head()

: 

In [ ]:
## Add churn ratio
customer_df["churn_ratio"]=customer_df["recency"]/customer_df["median_purchase_days"]
customer_df.head()

: 

In [ ]:
## Look at data to see if need to log monetary or other skews
customer_df.plot.scatter(x="frequency", y="monetary", color="blue")
plt.show() ## Probably, but transformation looks strange and want interpretation to be clean for client

: 

In [ ]:
## Create the kmeans df with normalization
customer_df['monetary_log'] = np.log1p(customer_df['monetary'])
kmeans_df=customer_df[["recency","frequency","monetary_log"]]
kmeans_df.head()

## Normalize the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(kmeans_df)

## Determine k parameter via elbow method
from sklearn.cluster import KMeans
inertia = []
for k in range(2, 8):
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    km.fit(X)
    inertia.append(km.inertia_)

: 

In [ ]:
## Elbow plot
ks = list(range(2, 8))
plt.figure(figsize=(6,4))
plt.plot(ks, inertia, marker='o')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method for Optimal k')
plt.xticks(ks)
plt.grid(True)
plt.show() # Chose 3 where drop off occurs

: 

In [ ]:
## Fit kmeans with 4 clusters
kmeans = KMeans(
    n_clusters=3,
    random_state=42,
    n_init=10
)

result = kmeans.fit_predict(X) ## Array of cluster assignments


: 

In [ ]:
## Determine segment names for the customer data frame
customer_df["cluster_assignment"]=result
customer_df.head()

: 

In [ ]:
## Make inferences about clusters
cluster_summary = (
    customer_df
    .groupby('cluster_assignment')
    .agg(
        avg_recency=('recency', 'mean'),
        avg_frequency=('frequency', 'mean'),
        avg_monetary=('monetary', 'mean'),
        customers=('customer_id', 'count')
    )
)
cluster_summary

: 

In [ ]:
## Customer segment assignment
cluster_map = {
    0: 'Seasonal Buyers',
    1: 'Monthly, High-Value Buyers',
    2: 'Experimental / Hesitant, Lower-Value Buyers'
}

customer_df['segment'] = customer_df['cluster_assignment'].map(cluster_map)
customer_df.head()

: 

In [ ]:
## Sanity check segmentation
customer_df.groupby('segment')['monetary'].sum()

: 

In [ ]:
## Visualize segmentation
import plotly.express as px

fig=px.scatter(
    customer_df,
    x='frequency',
    y='monetary',
    color='segment',
    size='recency',
    hover_data=["customer_id"]
)
# Increase figure size and make axes readable
fig.update_layout(
    width=900,
    height=600,
    title='Customer Segmentation (RFM)',
    xaxis_title='Frequency',
    yaxis_title='Monetary (£)',
    font=dict(size=14)
)

fig.show()

: 

In [ ]:
## Read the data back to CSV
customer_df.to_csv("../data/processed/customer_df.csv")
invoice_df.to_csv("../data/processed/invoice_df.csv")
full_df.to_csv("../data/processed/full_df.csv")

: 

: 